# Simple MNIST convnet

**Author:** [fchollet](https://twitter.com/fchollet)<br>
**Date created:** 2015/06/19<br>
**Last modified:** 2020/04/21<br>
**Description:** A simple convnet that achieves ~99% test accuracy on MNIST.

## Setup

In [1]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

2023-03-15 15:13:26.358605: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Prepare the data

In [2]:
# Model / data parameters
num_classes = 10
input_shape = (28, 28, 1)

# Load the data and split it between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

# Scale images to the [0, 1] range
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255
# Make sure images have shape (28, 28, 1)
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")


# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


## Build the model

In [10]:
# model = keras.Sequential(
#     [
#         keras.Input(shape=input_shape),
#         layers.Conv2D(32, kernel_size=(3, 3)),
#         layers.Activation("relu"),
#         layers.MaxPooling2D(pool_size=(2, 2)),
#         layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
#         layers.MaxPooling2D(pool_size=(2, 2)),
#         layers.Flatten(),
#         layers.Dropout(0.5),
#         layers.Dense(num_classes, activation="softmax"),
#     ]
# )

def get_resnet():
    inputs = keras.Input(shape=input_shape)
    x = layers.Conv2D(32, kernel_size=(3, 3))(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    
    res_1 = x
    x = layers.Conv2D(64, kernel_size=(1, 1))(x)
    
    x = layers.Conv2D(64, kernel_size=(3, 3), padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    
    x = layers.Conv2D(32, kernel_size=(3, 3), padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    
    x = layers.Add()([x, res_1])
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    
    x = layers.Conv2D(64, kernel_size=(3, 3), padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    
    res_2 = x
    
    x = layers.Conv2D(128, kernel_size=(3, 3), padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    
    x = layers.Conv2D(64, kernel_size=(3, 3), padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)
    
    x = layers.Add()([x, res_2])
    x = layers.MaxPooling2D(pool_size=(2, 2))(x)
    
    x = layers.Flatten()(x)
    x = layers.Dropout(0.5)(x)
    out = layers.Dense(num_classes, activation="softmax")(x)
    
    model = keras.Model(inputs=[inputs], outputs=[out])
    return model

model = get_resnet()
    

model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 conv2d_12 (Conv2D)             (None, 26, 26, 32)   320         ['input_4[0][0]']                
                                                                                                  
 batch_normalization_10 (BatchN  (None, 26, 26, 32)  128         ['conv2d_12[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_11 (Activation)     (None, 26, 26, 32)   0           ['batch_normalization_10[0]

## Train the model

In [9]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.1)

Epoch 1/15
422/422 [==============================] - 15s 22ms/step - loss: 1.0536 - accuracy: 0.7669 - val_loss: 0.4614 - val_accuracy: 0.8690
Epoch 2/15
422/422 [==============================] - 13s 32ms/step - loss: 0.1689 - accuracy: 0.9485 - val_loss: 0.1028 - val_accuracy: 0.9680
Epoch 3/15
422/422 [==============================] - 11s 27ms/step - loss: 0.1053 - accuracy: 0.9674 - val_loss: 0.0487 - val_accuracy: 0.9862
Epoch 4/15
422/422 [==============================] - 13s 31ms/step - loss: 0.0796 - accuracy: 0.9759 - val_loss: 0.0349 - val_accuracy: 0.9890
Epoch 5/15
422/422 [==============================] - 9s 22ms/step - loss: 0.0643 - accuracy: 0.9797 - val_loss: 0.1015 - val_accuracy: 0.9685
Epoch 6/15
422/422 [==============================] - 8s 19ms/step - loss: 0.0545 - accuracy: 0.9838 - val_loss: 0.0291 - val_accuracy: 0.9892
Epoch 7/15
422/422 [==============================] - 8s 19ms/step - loss: 0.0455 - accuracy: 0.9857 - val_loss: 0.0282 - val_accuracy: 0.

## Evaluate the trained model

In [5]:
score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])

Test loss: 0.026087595149874687
Test accuracy: 0.9918000102043152
